In [1]:
import os
import sys

os.chdir("C:/dataanalytics/python")
os.curdir

#Configure the environment . Set this up to the directory where spark is installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\\spark'
    
#create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exists. The names might change as
#versions change
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.4-src.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
 
#Initialize a spark context
from pyspark import SparkContext
from pyspark import SparkConf

#optionally configure spark
conf = SparkConf()
conf.setAppName("Jeffwiz")

#Initalize spark context onl runs once
sc = SparkContext('local', conf=conf)

In [8]:
sc.version #'2.2.0'
sc.pythonVer#'3.5'
sc.master#'local'
str(sc.sparkHome)#None
str(sc.sparkUser())#jeffnerd
sc.appName#
sc.applicationId
sc.defaultParallelism
sc.defaultMinPartitions


1

In [15]:
sc.appName

'Jeffwiz'

In [3]:
rdd = sc.parallelize([('a',7),('a',2),('b',2)])
rdd2 = sc.parallelize([('a',2),('d',1),('b',1)])
rdd3 = sc.parallelize(range(100))
rdd4 = sc.parallelize([("a",["x","y","z"]),
("b",["p", "r"])]).collect()

In [23]:
rdd.getNumPartitions() #1
rdd.count() #
rdd.countByKey()#defaultdict(int, {'a': 2, 'b': 1})
rdd.countByValue()#defaultdict(int, {('a', 2): 1, ('a', 7): 1, ('b', 2): 1})
rdd.collectAsMap()#{'a': 2, 'b': 2}
rdd3.sum()#4950

4950

In [27]:
rdd.collectAsMap()

{'a': 2, 'b': 2}

In [ ]:
rdd3.max() #99
rdd3.min()#0
rdd3.mean()#49.5
rdd3.stdev()#28.86607004772212
rdd3.variance()#833.25
rdd3.histogram()#(0, 33, 66, 99], [33, 33, 34])
rdd3.stats()#(count: 100, mean: 49.5, stdev: 28.86607004772212, max: 99.0, min: 0.0)

In [36]:
rdd3.stats()

(count: 100, mean: 49.5, stdev: 28.86607004772212, max: 99.0, min: 0.0)

In [37]:
#applying functions
rdd.map(lambda x: x + (x[1], x[0])).collect()

[('a', 7, 7, 'a'), ('a', 2, 2, 'a'), ('b', 2, 2, 'b')]

In [6]:
rdd5 = rdd.flatMap(lambda x: x + (x[1],x[0]))
rdd5.collect()

['a', 7, 7, 'a', 'a', 2, 2, 'a', 'b', 2, 2, 'b']

In [8]:
rdd4.flatMapValues(lambda x: x).collect()

AttributeError: 'list' object has no attribute 'flatMapValues'

In [ ]:
rdd.collect()#[('a', 7), ('a', 2), ('b', 2)]
rdd.take(2)#('a', 7), ('a', 2)]
rdd.first()#('a', 7)
rdd.top()#[('b', 2)]

In [15]:
rdd.top(1)

[('b', 2)]

In [16]:
#sampling
rdd3.sample(False, 0.15, 81).collect()

[3, 4, 27, 28, 35, 41, 43, 49, 53, 58, 85, 93]

In [ ]:
#Filtering
rdd.filter(lambda x: "a" in x).collect()#[('a', 7), ('a', 2)]
rdd.distinct().collect()#[('a', 2), ('b', 2), ('a', 7)]
rdd.keys().collect()#['a', 'a', 'b']

In [22]:
rdd.keys().collect()

['a', 'a', 'b']

In [25]:
def g(x):print(x)
rdd.foreach(g)

In [28]:
rdd.reduceByKey(lambda x,y : x + y).collect()

[('a', 9), ('b', 2)]

In [31]:
rdd.reduce(lambda a,b: a+b)

('a', 7, 'a', 2, 'b', 2)

In [32]:
#grouping by
rdd3.groupBy(lambda x: x%2).mapValues(list).collect()

[(0,
  [0,
   2,
   4,
   6,
   8,
   10,
   12,
   14,
   16,
   18,
   20,
   22,
   24,
   26,
   28,
   30,
   32,
   34,
   36,
   38,
   40,
   42,
   44,
   46,
   48,
   50,
   52,
   54,
   56,
   58,
   60,
   62,
   64,
   66,
   68,
   70,
   72,
   74,
   76,
   78,
   80,
   82,
   84,
   86,
   88,
   90,
   92,
   94,
   96,
   98]),
 (1,
  [1,
   3,
   5,
   7,
   9,
   11,
   13,
   15,
   17,
   19,
   21,
   23,
   25,
   27,
   29,
   31,
   33,
   35,
   37,
   39,
   41,
   43,
   45,
   47,
   49,
   51,
   53,
   55,
   57,
   59,
   61,
   63,
   65,
   67,
   69,
   71,
   73,
   75,
   77,
   79,
   81,
   83,
   85,
   87,
   89,
   91,
   93,
   95,
   97,
   99])]

In [33]:
rdd.groupByKey() .mapValues(list).collect()

[('a', [7, 2]), ('b', [2])]

In [ ]:
seqOp = (lambda x,y: (x[0]+y,x[1]+1))
combOp = (lambda x,y:(x[0]+y[0],x[1]+y[1]))
rdd3.aggregate((0,0),seqOp,combOp) #(4950,100) 
rdd.aggregateByKey((0,0),seqop,combop).collect()#[('a',(9,2)), ('b',(2,1))]
rdd3.fold(0,add) #4950 
rdd.foldByKey(0, add).collect()#Merge the values for each key[('a',9),('b',2)]
rdd3.keyBy(lambda x: x+x) .collect()

In [36]:
rdd3.aggregate((0,0),seqOp,combOp)

(4950, 100)

In [39]:
rdd.aggregateByKey((0,0),seqOp,combOp).collect()

[('a', (9, 2)), ('b', (2, 1))]

In [43]:
rdd.foldByKey(0, add)

NameError: name 'add' is not defined

In [44]:
rdd.subtract(rdd2).collect()

[('b', 2), ('a', 7)]

In [46]:
rdd2.subtractByKey(rdd).collect()

[('d', 1)]

In [47]:
rdd.cartesian(rdd2).collect()

[(('a', 7), ('a', 2)),
 (('a', 7), ('d', 1)),
 (('a', 7), ('b', 1)),
 (('a', 2), ('a', 2)),
 (('a', 2), ('d', 1)),
 (('a', 2), ('b', 1)),
 (('b', 2), ('a', 2)),
 (('b', 2), ('d', 1)),
 (('b', 2), ('b', 1))]

In [49]:
#sorting
rdd.sortBy(lambda x:x[1]).collect()

[('a', 2), ('b', 2), ('a', 7)]

In [51]:
rdd2.sortByKey().collect()

[('a', 2), ('b', 1), ('d', 1)]